<a href="https://colab.research.google.com/github/FrancescoMorri/Language_Classifcation/blob/main/Notebooks/Model_%26_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Define here your **PATH**:

In [92]:
#this if you work with Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [93]:
PATH = "/content/drive/MyDrive/language"

## Importing all the libraries and the class of the dataset

In [94]:
import torch
import torchvision
from torch.utils.data import Dataset
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
import time
from torchsummary import summary
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR
import numpy as np

In [95]:
class WordsDataset(Dataset):

    def __init__(self):
        self.samples = []

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        input = self.samples[idx][0]
        label = self.samples[idx][1]
        sample = [input, label]
        return sample
    
    def __addsample__(self, spelling, lexicality, max_length=22):
        if (type(spelling) == type("str")):
            input = torch.tensor(word_to_vec(spelling, max_length=max_length))
            label = torch.tensor(label_maker(lexicality))
            self.samples.append([input, label])
        else:
            print("Something Strange:", end='\t')
            print(spelling)

    def __removesample__(self, idx=0, value=None):
        '''
        If value is something, the element corresponding to that value is removed.
        Else the element at index idx is popped.
        '''
        if (value is not None):
            self.samples.remove(value)
        else:
            self.samples.pop(0)
        

# Loading Dataset and Creating Network Model

In [ ]:
trainloader = torch.load(PATH+"/trainloader.pth")
testloader = torch.load(PATH+"/testloader.pth")

In [103]:
class Words_Net(nn.Module):
    def __init__(self):
        super().__init__()

        self.linear1 = nn.Linear(in_features=598, out_features=1024)
        self.drop1 = nn.Dropout(p=0.2)

        self.linear2 = nn.Linear(in_features=1024, out_features=256)
        self.drop2 = nn.Dropout(p=0.2)

        self.linear3 = nn.Linear(in_features=256, out_features=128)
        self.drop3 = nn.Dropout(p=0.2)

        self.linear4 = nn.Linear(in_features=128, out_features=2)

        self.sig = nn.Sigmoid()

    def forward(self, x):
        x = torch.tensor(x)
        out = self.linear1(x)
        out = self.drop1(out)
        out = self.sig(out)

        out = self.linear2(out)
        out = self.drop2(out)
        out = self.sig(out)

        out = self.linear3(out)
        out = self.drop3(out)
        out = self.sig(out)

        out = self.linear4(out)

        return out

For the training we will use the GPU, even though is a fairly small network.

In [104]:
net = Words_Net()
want_cuda = True
have_cuda = torch.cuda.is_available()
if want_cuda and have_cuda:
    net.cuda()
    print(torch.cuda.get_device_name())
else:
    print ("No cuda available!\n")
summary(net, (598,))

No cuda available!

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 1024]         613,376
           Dropout-2                 [-1, 1024]               0
           Sigmoid-3                 [-1, 1024]               0
            Linear-4                  [-1, 256]         262,400
           Dropout-5                  [-1, 256]               0
           Sigmoid-6                  [-1, 256]               0
            Linear-7                  [-1, 128]          32,896
           Dropout-8                  [-1, 128]               0
           Sigmoid-9                  [-1, 128]               0
           Linear-10                    [-1, 2]             258
Total params: 908,930
Trainable params: 908,930
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.03
Params size (M

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


We define the training parameters, in this case we will use the CrossEntropy loss and the SGD algorithm to train the network.

In [105]:
EPOCH = 1
learn = 0.1

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learn, weight_decay = 4e-3)
#scheduler = optim.lr_scheduler.MultiStepLR(optimizer, [300,600], gamma=0.1)

The training function will return the loss of that epoch, mediated over the iteration on the dataset. It may also be added the option to acquire interesting data.

In [106]:
def training(acquire = False , PATH = None):
    running_loss = 0.0
    losst = 0
    index = 0
    for i, data in enumerate(trainloader, 0):
        # get the inputs, maybe they need to be tensors?
        inputs, labels = data
        #print(labels.shape)
        if want_cuda and have_cuda:
          inputs = inputs.cuda()
          labels = labels.cuda()

        outputs = net(inputs)

        optimizer.zero_grad()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        losst +=loss.item()
        index +=1
    return losst/index

Now we can do the actual training of the network.

In [107]:
start =time.time()
graph_data = []
mean_loss = 0
for epoch in range(EPOCH):
    loss = training()    
    print("Epoch: ", epoch, " Loss: %.10f"%(loss))
    mean_loss += loss
    graph_data.append((epoch, loss))
    np.savetxt( PATH+"/graphs/loss_data.csv", graph_data, delimiter=',')
    
    if (epoch % 50 == 49):
        #net.eval()
        #y = validate()
        #val_graph = np.append(val_graph, [[epoch, y]], axis=0)
        print("Mean loss: %.10f"%(mean_loss/50))
        mean_loss = 0
        print('Estimated time: %.3f min' %((EPOCH- epoch)*(time.time() - start)/(60*epoch)) )
        #torch.save(net.state_dict(), "/content/drive/My Drive/Saved_Sets/esperimento_7/32b/withLR2e-5/32_lr2e5_1000")
    #scheduler.step()
    net.train()
        
elapsed_time = time.time() - start
print('Finished Training (elapsed time %.3f min)' %(elapsed_time/60))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Epoch:  0  Loss: 0.7036838910
Finished Training (elapsed time 0.293 min)


In [ ]:
correct = 0
total = 0
vloss = 0
with torch.no_grad():
    for i, data in enumerate(testloader, 0):
        inputs, labels = data
        if want_cuda and torch.cuda.is_available():
            inputs = inputs.cuda()
            labels = labels.cuda()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print("Accuracy: ", round(correct/total *100, 4), "%")